<a href="https://colab.research.google.com/github/Sukrit888/AI-ML-Healthcare-/blob/main/Code_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 6.4 MB/s eta 0:00:00


In [ ]:
import boto3

# Configure AWS credentials
aws_access_key_id = 'YOUR AWS ACCESS KEY ID'
aws_secret_access_key = 'YOUR SECRET AWS ACCESS KEY'
region_name = 'YOUR REGION'

# Create a session
session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)

# Access S3
s3 = session.resource('s3')
bucket = s3.Bucket('pdf-input-bucket')

In [ ]:
import boto3

# Initialize Textract client
textract = boto3.client('textract',
                        aws_access_key_id='YOUR AWS ACCESS KEY ID',
                        aws_secret_access_key='YOUR SECRET AWS ACCESS KEY',
                        region_name='YOUR REGION')

# Start Document Analysis
response = textract.start_document_analysis(
    DocumentLocation={'S3Object': {'Bucket': 'pdf-input-bucket', 'Name': 'SampleEOB5.pdf'}},
    FeatureTypes=["FORMS"]
)
job_id = response['JobId']


In [ ]:
import time

def check_job_status(job_id):
    while True:
        response = textract.get_document_analysis(JobId=job_id)
        status = response['JobStatus']
        if status in ['SUCCEEDED', 'FAILED']:
            break
        time.sleep(5)
    return response

response = check_job_status(job_id)


In [ ]:
import boto3

# Initialize S3 client
s3 = boto3.client('s3',
                  aws_access_key_id='YOUR AWS KEY ID',
                  aws_secret_access_key='YOUR AWS SECRET ACCESS KEY',
                  region_name='YOUR REGION')

# List all buckets
response = s3.list_buckets()
buckets = [bucket['Name'] for bucket in response['Buckets']]
print("Existing buckets:", buckets)

Existing buckets: ['pdf-input-bucket', 'pdf-output-bucket']


In [ ]:
# Create output bucket if it doesn't exist
output_bucket_name = 'pdf-output-bucket'
region = 'us-east-1'  # Ensure this is the same region as your input bucket

# Create the bucket
try:
    s3.create_bucket(
        Bucket=output_bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
    )
    print(f"Bucket {output_bucket_name} created successfully.")
except s3.exceptions.BucketAlreadyOwnedByYou:
    print(f"Bucket {output_bucket_name} already exists.")
except Exception as e:
    print(f"Error creating bucket: {e}")

Error creating bucket: An error occurred (InvalidLocationConstraint) when calling the CreateBucket operation: The specified location-constraint is not valid


In [ ]:
import boto3
import json
import time

# Initialize Textract client
textract = boto3.client('textract',
                        aws_access_key_id='YOUR AWS ACCESS KEY ID',
                        aws_secret_access_key='YOUR AWS SECRET ACCESS KEY',
                        region_name='YOUR REGION')

# Start Document Analysis
response = textract.start_document_analysis(
    DocumentLocation={'S3Object': {'Bucket': 'pdf-input-bucket', 'Name': 'SampleEOB5.pdf'}},
    FeatureTypes=["FORMS"]
)
job_id = response['JobId']
print("Job started with ID:", job_id)

# Wait for the job to complete
def check_job_status(job_id):
    while True:
        response = textract.get_document_analysis(JobId=job_id)
        status = response['JobStatus']
        if status in ['SUCCEEDED', 'FAILED']:
            return response
        time.sleep(5)

response = check_job_status(job_id)
print("Job completed with status:", response['JobStatus'])

if response['JobStatus'] == 'SUCCEEDED':
    # Check if 'Blocks' is in the response
    if 'Blocks' in response:
        # Extract form data
        blocks = response['Blocks']
        form_data = [block for block in blocks if block['BlockType'] == 'KEY_VALUE_SET']

        # Convert to JSON
        form_data_json = json.dumps(form_data)

        # Initialize S3 resource
        s3 = boto3.resource('s3',
                            aws_access_key_id='YOUR AWS ACCESS KEY ID',
                            aws_secret_access_key='YOUR AWS SECRET ACCESS KEY',
                            region_name='YOUR REGION')

        # Save JSON to output S3 bucket
        output_bucket_name = 'pdf-output-bucket'
        output_file_key = 'extracted_form_data.json'

        try:
            s3.Object(output_bucket_name, output_file_key).put(Body=form_data_json)
            print(f"JSON data saved to {output_bucket_name}/{output_file_key}")
        except Exception as e:
            print(f"Error saving JSON data: {e}")
    else:
        print("The response does not contain 'Blocks'")
else:
    print(f"Textract job failed with status: {response['JobStatus']}")


Job started with ID: f261915b5a9afe4d5db1e8d83f928d05e419739f8d1915f0324d9e30d82e95a4
Job completed with status: SUCCEEDED
JSON data saved to pdf-output-bucket/extracted_form_data.json
